# Analyse des tendances pour conseiller des développeurs de jeux vidéos

Nous nous sommes demandés, si, grâce aux méthodes vu en cours, il était possible de connaître les tendances des jeux
vidéos, et de conseiller aux développeurs des bonnes pratiques pour réaliser de bon chiffres de ventes.

Malheureusement, les données que nous possédons ne montrent que peu d'informations sur les jeux en questions.
Cependant, nous pouvons tout de même les utiliser pour connaître les genres et les plateformes (console) qui ont le plus
de succès.

## I - Re-traitement des données

La première étape consiste à générer un nouveau dataset à partir de deux autres.
Le premier dataset, "DATA/vgsales" contient la plupart des informations dont nous avons besoins.
Le second ("DATA/steam.csv") contient plus de jeux et plus de variables sur eux, mais ne concerne que les jeux vendus
sur steam, et ne concerne donc qu'une seule plateforme. Cependant, il contient tout de même les dates de sortie de
certains jeux qui sont inconnues dans le dataset initial (pour lesquels la valeur de la date vaut "N/A").
Nous avons donc cherché à croiser ces données pour obtenir un troisième dataset.

Nous profitons de la création de ce troisième dataset pour supprimer les collones que nous considérons comme inutiles,
ou pour supprimer les lignes qui contiennent des données inconnues.

Le code correspondant à cette étape se trouve dans le fichier "generate_data.py". Il consiste à générer un code .csv,
qui est déjà généré. Il n'est donc pas nécessaire d'exécuter la cellule ci-dessous.

In [ ]:
import os

os.system('python3 generate_data.py')

Maintenant que notre dataset a été généré, nous pouvons le charger.
Pour cette partie du projet, nous ne prendrons que les jeux qui ont une date postérieure à 2012.
En effet, l'objectif de cette partie est de conseiller des développeurs pour le développement d'un nouveau jeux.

Les tendances antérieures à 2012 n'ont que peut d'intérêt pour nous.

In [19]:
import csv

data = []
with open('DATA/data_output.csv', mode='r') as infile:
    columns_keys = None
    reader = csv.reader(infile)

    for row in reader:
        if columns_keys is None:
            columns_keys = row
        else:
			elt = {}
			for index, _ in enumerate(columns_keys):
				elt[columns_keys[index]] = row[index]

			# Ici, on filtre les données
			if int(elt["Year"]) >= 2012:
				data.append(elt)
print(data)

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-19-84cc0f4f98a5>, line 12)

## III - Analyse en utilisant les fréquents et fermés.

Une méthode pour trouver les patterns qui ont le plus de succès, est de connaitre pour chaque motif de catégories,
les ventes moyennes de ces derniers, et de récupérer les motifs qui ont le meilleur score.

Cependant, certaines variables ne nous intéressent pas. En effet, il n'est pas pertinent de savoir qu'un jeux en
particulier (si le motif contient un titre) a eu beaucoup de succès.

Ensuite, il est intéressant de conserver des fréquents qui ont un support minimum, pour éviter que les moyennes que nous
réalisons soient basés sur peu d'apparitions, et soit donc moins fiables.

Enfin, nous allons chercher à ne conserver que les fermés. Cela nous permet d'afficher pour chaque motif l'ensemble
des caractéristiques qui ont influencées ce résultat.

In [3]:
import csv

loading dataset ...
Motif: ('Racing', 'Nintendo'), average_sales: 4.089189189189188, support: 37
Motif: ('Sports', 'Nintendo'), average_sales: 3.963818181818181, support: 55
Motif: ('Platform', 'Nintendo'), average_sales: 3.8394594594594595, support: 111
Motif: ('Shooter', 'Microsoft Game Studios'), average_sales: 3.4092857142857143, support: 28
Motif: ('Wii', '2006'), average_sales: 3.1343181818181822, support: 44
Motif: ('Simulation', 'Nintendo'), average_sales: 3.044642857142857, support: 28
Motif: ('Shooter', 'Nintendo'), average_sales: 2.90375, support: 24
Motif: ('Role-Playing', 'Nintendo'), average_sales: 2.7101904761904763, support: 105
Motif: ('GB',), average_sales: 2.622886597938143, support: 97
Motif: ('Nintendo',), average_sales: 2.563836206896549, support: 696
Motif: ('NES',), average_sales: 2.5619387755102028, support: 98
Motif: ('Action', 'Take-Two Interactive'), average_sales: 2.269677419354838, support: 93
Motif: ('2015', 'Shooter'), average_sales: 1.9455882352941174, 

Nous commençont par définir une fonction, qui pour chaque liste de paramètre, nous permettra de récupérer la liste des
motifs possibles pour cette liste.

In [ ]:
def getMotifs(line):
    """ Retourne tous les motifs d'une liste de valeurs donnée """
    # list to store all the sublists
    sublist = []
    for i in range(len(line) + 1):
        for j in range(i + 1, len(line) + 1):
            sli = line[i:j]  # make a slice of the subarray
            sublist.append(sli)  # add it to the list of sublists
    return sublist

Nous pouvons désormais parcourir notre dataset pour compter les suport des motifs présent dans notre dataset.

In [1]:
print("loading dataset ...")
frequents = {}
with open('DATA/data_output.csv', mode='r') as infile:
    columns_keys = None
    data = []
    reader = csv.reader(infile)

    for row_index, row in enumerate(reader):
        if columns_keys is None:
            columns_keys = row
        else:
            motifs = getMotifs(row[:-1])  # remode globals sales that will be our value
            global_sales = row[-1]

            for motif in motifs:
                key = tuple(motif)
                if key not in frequents:
                    frequents[key] = {
                        "total_sales": float(global_sales),
                        "index_apparitions": [row_index],
                        "support": 1
                    }
                else:
                    frequents[key]["total_sales"] += float(global_sales)
                    frequents[key]["support"] += 1
					frequents[key]["index_apparitions"].append(row_index)

# Filtrage des fréquents
minsup = 20
filtered_frequents = []  # Nous créons une liste afin de pouvoir la filtrer
for motif, content in frequents.items():
    if content["support"] >= minsup:
        frequent = {
            "motif": motif,
            "average_sales": frequents[motif]["total_sales"] / frequents[motif]["support"],
            "total_sales": frequents[motif]["total_sales"],
            "support": frequents[motif]["support"]
        }
        filtered_frequents.append(frequent)

# Maintenant que nous avons des fréquents, nous pouvons les triers pour trouver ceux qui peuvent intéresser les
# développeurs de jeux vidéos, c'est à dire, les motifs de jeux qui se vendent le mieux

filtered_frequents.sort(key=lambda frequent: frequent["average_sales"], reverse=True)

for frequent in filtered_frequents:
	print("Motif: " + str(frequent["motif"]) + ", average_sales: " + str(frequent["average_sales"]) + ", support: " + str(frequent["support"]))

loading dataset ...


NameError: name 'csv' is not defined